In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


선형 회귀는 알려진 다른 관련 데이터 값을 사용하여 알 수 없는 데이터의 값을 예측하는 데이터 분석 기법입니다.  
알 수 없는 변수 또는 종속 변수와 알려진 변수 또는 독립 변수를 선형 방정식으로 수학적으로 모델링합니다.

학습 데이터를 만든다.

In [2]:
xData = [1, 2, 3, 4, 5, 6, 7] # 근무 시간, 데이터
yData = [25000, 55000, 75000, 110000, 128000, 155000, 180000] # 근무 시간에 따른 매출 금액, 실제값, 관측값, 답, 레이블

$y = ax + b$를 만족시키는 $a$(기울기, 가중치)와 $b$(y절편, 바이어스)를 정해야 한다.  
과적합을 방지하기 위해서 난수를 발생시켜 기울기와 y절편을 정한다. 과적합이란 학습시킨 데이터에서는 높은 정확도를 보이지만 학습시킨 데이터 이외의 데이터에서는 낮은 정확도를 보이는 문제점을 말한다.

In [3]:
# random_uniform() 메소드로 최소값과 최대값 사이에서 지정한 개수의 난수를 발생시킨다.
# random_uniform([난수의 개수], 최소값, 최대값)
a = tf.Variable(tf.random_uniform([1], -100, 100)) # 기울기, -100 ~ 100 사이의 난수
b = tf.Variable(tf.random_uniform([1], -100, 100)) # y절편, -100 ~ 100 사이의 난수
print('a = {}, b = {}'.format(a, b))

a = <tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>, b = <tf.Variable 'Variable_1:0' shape=(1,) dtype=float32_ref>


In [4]:
# tensorflow 변수를 초기화하고 난수로 발생시킨 기울기와 y절편을 확인한다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a = {}, b = {}'.format(sess.run(a), sess.run(b)))

a = [72.77243], b = [-86.70447]


근무 시간과 근무 시간에 따른 매출 금액을 기억할 placeholder를 선언한다.

In [5]:
x = tf.placeholder(dtype=tf.float32) # 근무 시간을 기억할 placeholder를 선언한다.
y = tf.placeholder(dtype=tf.float32) # 근무 시간에 따른 매출 금액을 기억할 placeholder를 선언한다.

1차 방정식 형태의 가설($y = ax + b$)을 세운다.

In [6]:
# 1차 방정식 형태의 가설, Y => 예측값
Y = a * x + b

오차(손실, 비용) 함수를 정의한다.

In [7]:
# 평균 제곱법을 이용하는 오차 함수
# 실제값(yData => y라는 placeholder)과 예측값(Y)의 편차에 대한 제곱의 평균을 이용한다.
# square() 메소드로 tensorflow에서 제곱값을 계산한다.
# reduce_mean() 메소드로 tensorflow에서 평균을 계산한다.
cost = tf.reduce_mean(tf.square(Y - y))

경사 하강법 알고리즘을 이용해서 오차 함수의 결과를 가장 작게만드는 방향으로 학습하도록 정의한다.

In [8]:
# 경사 하강법 알고리즘의 학습율(learning rate)을 설정한다.
learning_rate = tf.Variable(0.01)
# train.GradientDescentOptimizer() 클래스의 인수로 학습율을 넘겨서 경사 하강법 알고리즘을 이용해 
# 오차를 계산한다.
# minimize() 메소드의 인수로 오차 함수를 넘겨서 경사 하강법에 의해 계산되는 오차의 최소값을 찾는다.
train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

학습시킨다.

In [9]:
# 세션을 만들고 변수를 초기화시킨다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(5001):
    # 오차 함수의 결과를 가장 작게 만드는 학습을 할 수 있도록 placeholder에 데이터를 대입하고 학습시킨다.
    sess.run(train, feed_dict={x: xData, y: yData})
    
    # 일정 작업 단위(epoch)마다 중간 결과를 출력한다.
    if i % 500 == 0:
        # epoch(i값), 오차(cost), 기울기(a), y절편(b)
        '''
        print('epoch: %5d, cost: %14.2f, a: %9.3f, b: %9.3f' % (
            i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a), sess.run(b))
        )
        '''
        print('epoch: {:5d}, cost: {:14.2f}, a: {:9.3f}, b: {:9.3f}'.format(
            i, sess.run(cost, feed_dict={x: xData, y: yData}), sess.run(a)[0], sess.run(b)[0])
        )
# ===== 학습 끝

print('최적의 기울기: {:9.3f}, 최적의 y절편: {:9.3f}'.format(sess.run(a)[0], sess.run(b)[0]))
print('8시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(sess.run(a)[0] * 8 + sess.run(b)[0]))

epoch:     0, cost:  4598764544.00, a: 10353.370, b:  2129.437
epoch:   500, cost:     8688943.00, a: 25535.742, b:  1958.992
epoch:  1000, cost:     8633843.00, a: 25627.248, b:  1505.866
epoch:  1500, cost:     8632683.00, a: 25640.582, b:  1439.840
epoch:  2000, cost:     8632658.00, a: 25642.525, b:  1430.212
epoch:  2500, cost:     8632654.00, a: 25642.805, b:  1428.823
epoch:  3000, cost:     8632657.00, a: 25642.844, b:  1428.634
epoch:  3500, cost:     8632657.00, a: 25642.848, b:  1428.613
epoch:  4000, cost:     8632657.00, a: 25642.848, b:  1428.613
epoch:  4500, cost:     8632657.00, a: 25642.848, b:  1428.613
epoch:  5000, cost:     8632657.00, a: 25642.848, b:  1428.613
최적의 기울기: 25642.848, 최적의 y절편:  1428.613
8시간 근무했을 때 예상 매출 금액: 206,571원


학습이 완료된 후 원하는 입력에 대한 예측 결과를 출력한다.

In [10]:
for i in range(8, 25):
    # print('{:2d}시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(i, sess.run(a)[0] * i + sess.run(b)[0]))
    print('{:2d}시간 근무했을 때 예상 매출 금액: {:,.0f}원'.format(i, sess.run(Y, feed_dict={x: [i]})[0]))

 8시간 근무했을 때 예상 매출 금액: 206,571원
 9시간 근무했을 때 예상 매출 금액: 232,214원
10시간 근무했을 때 예상 매출 금액: 257,857원
11시간 근무했을 때 예상 매출 금액: 283,500원
12시간 근무했을 때 예상 매출 금액: 309,143원
13시간 근무했을 때 예상 매출 금액: 334,786원
14시간 근무했을 때 예상 매출 금액: 360,428원
15시간 근무했을 때 예상 매출 금액: 386,071원
16시간 근무했을 때 예상 매출 금액: 411,714원
17시간 근무했을 때 예상 매출 금액: 437,357원
18시간 근무했을 때 예상 매출 금액: 463,000원
19시간 근무했을 때 예상 매출 금액: 488,643원
20시간 근무했을 때 예상 매출 금액: 514,286원
21시간 근무했을 때 예상 매출 금액: 539,928원
22시간 근무했을 때 예상 매출 금액: 565,571원
23시간 근무했을 때 예상 매출 금액: 591,214원
24시간 근무했을 때 예상 매출 금액: 616,857원
